<h1> OQC Challenge </h1>

<h2>James Benstead, February 2022</h2>


The work to meet this challenge will be presented in this single Jupyter notebook for simplicity. Any code will be written in PYTHON3.
Unfortunately, I did not have the time to complete the challenge as I'd intended, but hopefully what I had planned will be clear and also correct.

<h2>Task 1</h2>

For this first task I created a short piece of python code that takes in a set of user supplied single qubit gates with arbitrary rotation angles and calculates what their sequential effect will be on an arbitrary input state $\vert\psi>$. 
$$|\psi>=\alpha|0>+\beta|1>$$ where $\alpha$ and $\beta$ may be complex coefficients.

The user inputs values for $\alpha$ and $\beta$ and these are normalised such that $\alpha^2 + \beta^2 = 1$.This normalisation is checked by the code.

In [3]:
import numpy as np
import cmath as cm
import math

# declare pi and i
pi = cm.pi
imagj = complex(0,1)

# define a random state vector
# THIS WOULD BE INPUT, EITHER BY A USER OR SOME OTHER CODE
# a and b are coefficients for |0> and |1> states respectively
a = complex(1.,0)
b = complex(0.5,0)
stateI = np.matrix([[a],[b]])
# coefficients are normalised here so that |state vector|=1
absolute = cm.sqrt((a*a)+(b*b))
stateI/=absolute
absolute = abs(stateI)
# Test of normalisation result is ouput as well as the normalised coefficients
print('Magnitude of state: '+str(absolute[0]**2+absolute[1]**2))
print("Initial state is: ")
print(stateI)

Magnitude of state: [[1.]]
Initial state is: 
[[0.89442719+0.j]
 [0.4472136 +0.j]]


The user next supplies a set of gate operations to be performed on this state.

In [4]:
# create an input list of gate operations
# THIS WOULD BE INPUT, EITHER BY A USER OR SOME OTHER CODE
input_gates = [['X',90],['X',90],['Z',90],['Y',30], ['Z',120],['Y',20]]

Then each of these gates is run sequentially. Here 'X' really means R_x (arbitrary X rotation) and so on with Y and Z. Rather than use an existing function for each of these gates I created my own functions for these operations. Unfortunately I got a little carried away with this, which ate into my challenge time... The functions are directly below, followed by the loop over the gates.

In [5]:
# create my own functions to perform abitrary X, Y and Z rotations
def xgate(state1,angle):
    imagj = complex(0,1)
    xmat = np.array([[cm.cos(angle/2.),-1.*imagj*cm.sin(angle/2.)],[-1.*imagj*cm.sin(angle/2.),cm.cos(angle/2.)]])
    xmat*=imagj
    state2 = np.matmul(xmat,state1)
    return state2
  
def ygate(state1,angle):
    imagj = complex(0,1)
    ymat = np.array([[cm.cos(angle/2.),-1*cm.sin(angle/2.)],[cm.sin(angle/2.),cm.cos(angle/2.)]])
    ymat*=imagj
    state2 = np.matmul(ymat,state1)
    return state2

def zgate(state1,angle):
    imagj = complex(0,1)
    zmat = np.array([[-1.*cm.exp(imagj*angle*.5),0.],[0.,cm.exp(imagj*angle*.5)]])
    zmat*=imagj
    state2 = np.matmul(zmat,state1)
    return state2
# Simple degrees to radians converter  
def degToRads(degrees):
    radians = degrees *  math.pi / 180.
    return radians

In [6]:
# Now going through each gate and rotating our initial state
inputGateNumber = 1
stateTemp = stateI
for gate in input_gates:
    print("Input gate "+str(inputGateNumber))
    inputGateNumber+=1
    print(gate)
    if gate[0] == 'X':
        stateTemp = xgate(stateTemp,degToRads(gate[1]))
    if gate[0] == 'Y':
        stateTemp = ygate(stateTemp,degToRads(gate[1]))
    if gate[0] == 'Z':
        stateTemp = zgate(stateTemp,degToRads(gate[1]))
stateF = stateTemp

# Outputting final state after rotations
print("Final state is: ", stateF)
# Check it is still normalised
absolute = abs(stateF)#stateF[0]**2+stateF[1]**2
print('Magnitude of state: '+str(absolute[0]**2+absolute[1]**2))

Input gate 1
['X', 90]
Input gate 2
['X', 90]
Input gate 3
['Z', 90]
Input gate 4
['Y', 30]
Input gate 5
['Z', 120]
Input gate 6
['Y', 20]
Final state is:  [[-0.5056296 -0.13548304j]
 [-0.82301362-0.22052583j]]
Magnitude of state: [[1.]]


This code was tested against known rotation results and seemed to perform correctly. 
Now, knowing what effect these rotations will have on an abitrary state, the task is to minimise these operations. Using some algebra I was able to calculate and then implement in my code, a way to find a single rotation matrix $M$ that replaces this sequence of gates.

$$M|\psi_{i}>=|\psi_{f}>$$
$$M=|\psi_{f}><\psi_{i}|$$
$M$ is created by the outer product of the initial and final states. This procedure works if $M$ and the gates it is replacing are unitary (which is also a test of my earlier X, Y and Z functions).


In [7]:
# NOW WORKING TO MINIMISE GATE ROTATIONS
# Have starting state and end state, so can create a single matrix to make the rotation between them
print('New matrix is:')
new_mat = np.outer(stateF,stateI)
print(new_mat)
print(' ')
# Test that this new matrix works in place of the previous set of sequential gates
stateF2 = np.matmul(new_mat,stateI)
print("Final state calculated using this new single matrix: ", stateF2)

New matrix is:
[[-0.45224886-0.12117972j -0.22612443-0.06058986j]
 [-0.73612576-0.1972443j  -0.36806288-0.09862215j]]
 
Final state calculated using this new single matrix:  [[-0.5056296 -0.13548304j]
 [-0.82301362-0.22052583j]]


It can be seen that this new single matrix performs the same rotation as that of the initial sequence of gates, but in a single rotation. Unfortunately, no such gate exists to perform this arbitrary single rotation, so this matrix must be decomposed into a sequence of $R_x$, $R_y$ and $R_z$ gates. 
The simplest construct will be a combination of two gates e.g. $R_y$ and $R_z$. Unfortunately I was not able to perform this decomposition...

<h2>Task 2

Unfortunately, given I did not complete the coding of Task 1 I was unable to do more than simply think about Task 2. With the absence of rotations about the Y axis, analagous rotations would need to be performed using e.g. the sequence $R_y(\theta)=R_z(90)R_x(\theta)R_z(-90)$. Unfortunately this would increase circuit depth and hence decoherence effects.

<h2>Task 3

Again, as with Task 2, this one was not attempted, but after thinking about it a few comments are;
- assuming a z rotation takes 10x that of an x rotation, whereever a z rotation appears it could be replaced by an equivalent set of rotations using different gates available.
- if y is available then a combination of x and y, otherwise single z gates can be replaced by HXH, assuming a Hadamard gate is available and also takes <=4x the length of an x gate to run.
- I would also add a safety margin between co-timed parallel operations to account for electronics "jitter". This jitter could be measured on the hardware used and input by the user before a calculation.

<h2>Task 4

Unfortunately I did not attempt Task 4.